In [2]:
import os
import pandas as pd
#import nltk
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize
import os
import re
#os.getcwd()

In [3]:
comentarios = pd.read_csv('C:/Users/Kikes/Documents/GitHub/NLP/df_comms.csv', encoding='utf-8', index_col=0)

In [4]:
comentarios_nn = comentarios[comentarios.comm_msg.notnull()]
#comentarios_nn[comentarios_nn.comm_id=='545801849092539_545952342410823']

In [5]:
#Desde acá: https://nlp.stanford.edu/software/CRF-NER.shtml#Download
#Se baja: https://nlp.stanford.edu/software/stanford-ner-2017-06-09.zip
#Se descomprime el .zip y a la carpeta que está adentro se le cambia el nombre de: stanford-ner-2017-06-09 a: stanford-ner 

#import os
#java_path = "C:/Program Files/Java/jre1.8.0_131/bin/java.exe"
java_path = 'C:/Program Files (x86)/Java/jre1.8.0_131/bin/java.exe'

os.environ['JAVAHOME'] = java_path

#os.getcwd()
#Hay que cambiar el directorio y que apunte donde está la carpeta que descomprimimos
#os.chdir("C:/Users/axmarr/Documents/Varios/maestria/NLP/")
os.chdir('C:/Users/Kikes/Documents/maestria/NLP/')

# -*- coding: utf-8 -*-

#from nltk.tag import StanfordNERTagger
#from nltk.tokenize import word_tokenize

st = StanfordNERTagger('stanford-ner/classifiers/spanish.ancora.distsim.s512.crf.ser.gz',
					   'stanford-ner/stanford-ner.jar',
					   encoding='utf-8')

In [9]:
def extraeNombres(comm):
    comm_str = str(comm)#comm.to_json()
    #comm_token = word_tokenize(comm_str)
    comm_token = word_tokenize(comm_str)
    #comm_token = word_tokenize(comm)
    comm_class = st.tag(comm_token)
    comm_list_per = [pers for pers in comm_class if pers[1] =='PERS']
    comm_per = re.sub(r'[\[\]\(\)\'\,]', "", str([i[0] for i in comm_list_per]))
    return comm_per

In [17]:
df = comentarios.head(5)

In [18]:
%%timeit

pd.options.mode.chained_assignment = None
df['nombres'] = df['comm_msg'].apply(lambda x: extraeNombres(x))
pd.options.mode.chained_assignment = 'warn'


#1 registro: 1 loop, best of 3: 1.73 s per loop
#5 registro: 1 loop, best of 3: 8.65 s per loop

1 loop, best of 3: 8.65 s per loop


In [10]:
#Ver como optimizar este punto
#https://engineering.upside.com/a-beginners-guide-to-optimizing-pandas-code-for-speed-c09ef2c6a4d6

pd.options.mode.chained_assignment = None
comentarios['nombres'] = comentarios['comm_msg'].apply(lambda x: extraeNombres(x))
pd.options.mode.chained_assignment = 'warn'

In [12]:
n_comm = len(comentarios)

for c in range(n_comm):
    ext_name = str(comentarios.nombres[c])
    ext_comm = str(comentarios.comm_msg[c])
    comm_no_name = ext_comm.replace(ext_name,'')
    comentarios.comm_msg[c] = comm_no_name


C:\Users\Kikes\Anaconda3\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [13]:
comentarios.head()

,post_id,comm_id,comm_msg,comm_date,emojis,emoji_count,nombres
0,262755230730537_546328889039835,546328889039835_546416972364360,Mmm que rico,2017-11-22 19:03:36,NaN,0,
1,262755230730537_546317689040955,546317689040955_546341119038612,No tienen una copa para postres que usan frasc...,2017-11-22 15:14:52,😵😂😂😂,4,
2,262755230730537_546317689040955,546317689040955_546422655697125,Mira para navidad vos veras con que reemplaz...,2017-11-22 19:26:31,😱,1,Melina Yasmin
3,262755230730537_546317689040955,546317689040955_546339792372078,...q rico noo!!?fresquito para el verano......,2017-11-22 15:09:30,😄,1,Majo Maciel
4,262755230730537_546317689040955,546317689040955_546346215704769,"Tendencia o no, si veo a un moso trayendo me u...",2017-11-22 15:39:09,NaN,0,


In [14]:
comentarios.to_csv( 'comentarios.csv', encoding = 'utf-8')